In [3]:
!head -3 /media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/20131228/part-19143

['2013-12-28 23:40:34', '8613770773315', '111.30.132.39', 'web', 'wspeed.qq.com', 'qq', '-', '1087', '3', '3', '8', '2148', '-1']
['2013-12-28 23:41:41', '8613770773315', '111.30.132.38', 'web', 'wspeed.qq.com', 'qq', '-', '1621', '5', '5', '8', '2148', '-1']
['2013-12-28 23:43:52', '8613770773315', '111.30.132.39', 'web', 'wspeed.qq.com', 'qq', '-', '1140', '5', '5', '8', '2148', '-1']


In [91]:
from collections import Counter


In [7]:
qq_5000_user_dpi_day1_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/20131228/part-*"
qq_5000_user_dpi_day2_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/20131229/part-*"
qq_5000_user_dpi_day3_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/20131230/part-*"

qq_5000_user_dpi_day4_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/20131231/part-*"

qq_5000_user_dpi_3days_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/qq_5000_user_dpi/3days/*/part-*"

In [128]:
# train data by day
qq_5000_user_dpi_day1 = sc.textFile(qq_5000_user_dpi_day1_f).coalesce(50)  # repartition
qq_5000_user_dpi_day2 = sc.textFile(qq_5000_user_dpi_day2_f).coalesce(50)
qq_5000_user_dpi_day3 = sc.textFile(qq_5000_user_dpi_day3_f).coalesce(50)

# test data
qq_5000_user_dpi_day4 = sc.textFile(qq_5000_user_dpi_day4_f).coalesce(50)

# all 3 days' train data
qq_5000_user_dpi_3days = sc.textFile(qq_5000_user_dpi_3days_f).coalesce(50)  

qq_5000_user_dpi_4days = sc.union([qq_5000_user_dpi_3days, qq_5000_user_dpi_day4])

In [10]:
train_positive_pairs_day1 = qq_5000_user_dpi_day1.map(lambda x: 'day1-' + eval(x)[1] + '-' + eval(x)[2])

In [11]:
train_positive_pairs_day1 = list(set(train_positive_pairs_day1.collect()))

In [12]:
print(len(train_positive_pairs_day1))
print(train_positive_pairs_day1[5:9])

76569
['day1-8613770303266-183.207.233.97', 'day1-8615105172269-117.135.171.159', 'day1-8613913813828-117.135.169.19', 'day1-8618252070231-211.136.233.104']


In [13]:
train_positive_pairs_day2 = qq_5000_user_dpi_day2.map(lambda x: 'day2-' + eval(x)[1] + '-' + eval(x)[2])

In [14]:
train_positive_pairs_day2 = list(set(train_positive_pairs_day2.collect()))

In [15]:
print(len(train_positive_pairs_day2))
print(train_positive_pairs_day2[5:9])

76809
['day2-8613851741572-120.204.200.164', 'day2-8613851840923-117.135.171.239', 'day2-8613813838348-120.196.210.95', 'day2-8615189842330-120.204.201.199']


In [17]:
train_positive_pairs_day3 = qq_5000_user_dpi_day3.map(lambda x: 'day3-' + eval(x)[1] + '-' + eval(x)[2])

In [18]:
train_positive_pairs_day3 = list(set(train_positive_pairs_day3.collect()))

In [19]:
print(len(train_positive_pairs_day3))
print(train_positive_pairs_day3[5:9])

82208
['day3-8615895979031-120.196.210.95', 'day3-8615895989367-120.198.196.11', 'day3-8613665722685-120.198.201.59', 'day3-8613951085451-60.172.80.57']


In [20]:
train_positive_pairs = train_positive_pairs_day1 + train_positive_pairs_day2 + train_positive_pairs_day3

In [21]:
print(len(train_positive_pairs))
print(train_positive_pairs[:3],train_positive_pairs[76809:76811])

235586
['day1-8618761696532-221.179.18.179', 'day1-8613851906441-221.179.19.156', 'day1-8618252050428-111.30.128.27'] ['day2-8618752026657-120.198.199.176', 'day2-8613815857018-101.226.76.175']


In [27]:
sc.parallelize(train_positive_pairs).coalesce(24)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/train_pos_pairs")

In [150]:
test_positive_pairs_day4 = list(set(qq_5000_user_dpi_day4.filter(lambda x: eval(x)[2] in server_ip_1684)\
                                    .map(lambda x: eval(x)[1] + '-' + eval(x)[2]).collect()))

In [22]:
# 5000个用户的id和1684个服务器ip
user_phone_5000_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/user_phone_5000/part-*"
server_ip_1684_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/server_ip_1684/part-*"

In [23]:
user_phone_5000 = sc.textFile(user_phone_5000_f).collect()
server_ip_1684 = sc.textFile(server_ip_1684_f).collect()

In [24]:
print(len(user_phone_5000),len(server_ip_1684))

5000 1684


In [117]:
all_edges = [] # 5000 * 1684
for u in user_phone_5000:
    for s in server_ip_1684:
        edge = u + '-' + s
        all_edges.append(edge)

In [118]:
print(len(all_edges))
print(all_edges[2:5])

8420000
['8615195898003-111.30.132.109', '8615195898003-117.135.130.175', '8615195898003-14.17.37.157']


In [32]:
train_positive_pairs_3days = qq_5000_user_dpi_3days.map(lambda x: eval(x)[1] + '-' + eval(x)[2]).collect()

In [33]:
unlabled_edges = list(set(all_edges)-set(train_positive_pairs_3days))   # 三天都没有过连接的user-server对作为unlabled数据

In [34]:
train_unlabled_pairs = ['day123-' + i for i in unlabled_edges]

In [35]:
print(len(train_unlabled_pairs))
print(train_unlabled_pairs[2:5])

8268837
['day123-8615151857865-117.135.171.164', 'day123-8613564790292-112.90.78.61', 'day123-8613914733776-61.177.126.84']


In [151]:
test_ublabled_pairs_day4 = list(set(all_edges)-set(test_positive_pairs_day4))

In [152]:
test_ublabled_pairs_day4 = ['day4-' + i for i in test_ublabled_pairs_day4]

In [153]:
print(len(test_positive_pairs_day4))
print(len(test_ublabled_pairs_day4),test_ublabled_pairs_day4[:3])

77653
8342347 ['day4-8615151840328-119.167.195.26', 'day4-8615850607902-203.205.161.165', 'day4-8615151857865-117.135.171.164']


In [154]:
77653+8342347

8420000

In [155]:
sc.parallelize(test_ublabled_pairs_day4).coalesce(24)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/test_unlabled_pairs/1231")

In [39]:
# 总共63个终端品牌
all_brand = list(set(qq_5000_user_dpi_3days.map(lambda x: eval(x)[12]).collect()))

In [40]:
print(len(all_brand))
print(all_brand)

63
['�ƽ�', '֪��Ѹ��', '����������', 'Rayhov', '����Ѷ', 'E��', '��������', 'TCL', 'LG', '���ͨ', '�¿�˹', 'ŷ����', '��Ѷ', '�λ���', '����Ħ��', '����', 'ɽկ��', 'oppo', '�е�', '���', '˫��', 'С�\u05ffƼ�', 'NEC', '���ᰮ����', 'UNISCOPE', '-1', 'ɣ��', '�Ϲ⺣̩', 'HTC', '���Ŵ�', '��Ψ־Զ', 'С��', '��ʱ��', 'ƻ��', 'OPPO', 'ŵ����', '��ͨ', '���Դ', 'ʱͨΰҵ', 'Э������', '�\ueeea', 'άͼ', '��ɭ�Ƽ�', '������ͨ', '��Τ��', '����ͨ', 'ά��', '������', '���մ�', '�Ź�ʱ��', 'ǧ����', '�ȷ�', 'Ӣ����', '�Ķ��Ƽ�', '�µϺ�', 'Ħ������', 'Ħ��', '��Դ', '��ݮ', 'ŷ��', '�ȸ�', '��Ψ', '��Ϊ']


In [43]:
sc.parallelize(all_brand).coalesce(1)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/all_brand")

In [44]:
def brand(x):
    all_brand = ['�ƽ�', '֪��Ѹ��', '����������', 'Rayhov', '����Ѷ', 'E��', '��������', 'TCL', 'LG', '���ͨ', '�¿�˹', 'ŷ����', '��Ѷ', '�λ���', '����Ħ��', '����', 'ɽկ��', 'oppo', '�е�', '���', '˫��', 'С�\u05ffƼ�', 'NEC', '���ᰮ����', 'UNISCOPE', '-1', 'ɣ��', '�Ϲ⺣̩', 'HTC', '���Ŵ�', '��Ψ־Զ', 'С��', '��ʱ��', 'ƻ��', 'OPPO', 'ŵ����', '��ͨ', '���Դ', 'ʱͨΰҵ', 'Э������', '�\ueeea', 'άͼ', '��ɭ�Ƽ�', '������ͨ', '��Τ��', '����ͨ', 'ά��', '������', '���մ�', '�Ź�ʱ��', 'ǧ����', '�ȷ�', 'Ӣ����', '�Ķ��Ƽ�', '�µϺ�', 'Ħ������', 'Ħ��', '��Դ', '��ݮ', 'ŷ��', '�ȸ�', '��Ψ', '��Ϊ']
    feature_brand = [0 for i in range(63)]
    feature_brand[all_brand.index(x)] = 1
    return feature_brand

In [36]:
user_brand_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/user_brand"

In [46]:
# ('8613851429132', '������')
user_brand = sc.textFile(user_brand_f).map(lambda x: (eval(x)[0], brand(eval(x)[1])))

In [48]:
print(user_brand.take(3))
print(user_brand.take(3))

[('8613851429132', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('8615722826043', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('8618260098095', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


In [49]:
user_brand.coalesce(2)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/user_brand")

In [52]:
train_positive_pairs_day1_user = sc.parallelize(train_positive_pairs_day1).map(lambda x:((x.split('-')[1],x)))

In [53]:
train_positive_pairs_day1_user.take(1)

[('8618761696532', 'day1-8618761696532-221.179.18.179')]

In [61]:
train_positive_pairs_day1_user_brand = train_positive_pairs_day1_user.union(user_brand).groupByKey().mapValues(list)

In [63]:
print(train_positive_pairs_day1_user_brand.take(2))

[('8613851429132', ['day1-8613851429132-111.30.135.151', 'day1-8613851429132-117.135.171.159', 'day1-8613851429132-117.135.169.19', 'day1-8613851429132-123.125.87.72', 'day1-8613851429132-223.85.133.151', 'day1-8613851429132-117.135.171.156', 'day1-8613851429132-140.206.160.179', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), ('8613805141374', ['day1-8613805141374-120.198.189.123', 'day1-8613805141374-120.204.200.161', 'day1-8613805141374-58.250.132.244', 'day1-8613805141374-120.198.189.124', 'day1-8613805141374-120.198.189.125', 'day1-8613805141374-183.207.234.69', 'day1-8613805141374-117.135.169.19', 'day1-8613805141374-58.251.62.24', 'day1-8613805141374-120.204.200.174', 'day1-8613805141374-183.207.234.71', 'day1-8613805141374-183.207.234.72', 'day1-8613805141374-58.250.132.81', 'day1-8613805141374-112.90.140.203', 'day1-8613805141374-120.

In [64]:
user_activetime_f = "/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/qq_5000_user_data/user_activetime" 

In [65]:
# 用户前三天每个小时的访问qq的次数总和.
# [('8613851429132', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 10, 4, 0, 12, 0, 0, 0, 0, 39, 8, 0, 0, 0])]
user_activetime = sc.textFile(user_activetime_f).map(lambda x:(eval(x)[0],eval(x)[1]))

In [68]:
train_positive_pairs_day1_user_brand_activetime = train_positive_pairs_day1_user.union(user_brand)\
.union(user_activetime).groupByKey().mapValues(list).map(lambda x: x[1])

In [70]:
print(train_positive_pairs_day1_user_brand_activetime.take(2))

[['day1-8613913304443-58.251.148.250', 'day1-8613913304443-58.251.148.164', 'day1-8613913304443-120.204.200.161', 'day1-8613913304443-111.30.135.151', 'day1-8613913304443-120.198.189.102', 'day1-8613913304443-117.135.128.17', 'day1-8613913304443-120.198.201.59', 'day1-8613913304443-117.135.171.156', 'day1-8613913304443-120.204.200.103', 'day1-8613913304443-120.198.189.104', 'day1-8613913304443-58.251.148.145', 'day1-8613913304443-117.135.128.15', 'day1-8613913304443-120.198.201.62', 'day1-8613913304443-183.207.233.95', 'day1-8613913304443-117.135.171.236', 'day1-8613913304443-117.135.128.31', 'day1-8613913304443-58.251.148.242', 'day1-8613913304443-58.251.148.246', 'day1-8613913304443-117.135.128.19', 'day1-8613913304443-183.207.233.97', 'day1-8613913304443-183.232.125.153', 'day1-8613913304443-117.135.128.13', 'day1-8613913304443-223.85.133.151', 'day1-8613913304443-120.204.200.164', 'day1-8613913304443-117.135.128.58', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [73]:
def brand_activetime(x):
    res = []
    for i in x[:-2]:
        s = (i, x[-2], x[-1])
        res.append(s)
    return res

In [80]:
aa = train_positive_pairs_day1_user_brand_activetime.map(lambda x: brand_activetime(x)).flatMap(lambda x: x)

In [82]:
print(aa.take(2))

[('day1-8613913304443-58.251.148.250', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 6, 40, 11, 0, 16, 31, 0, 16, 46, 19, 6, 19, 16, 51, 0, 0, 0]), ('day1-8613913304443-58.251.148.164', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 0, 0, 6, 40, 11, 0, 16, 31, 0, 16, 46, 19, 6, 19, 16, 51, 0, 0, 0])]


In [83]:
train_positive_pairs_day1_server = sc.parallelize(train_positive_pairs_day1).map(lambda x:((x.split('-')[2],x)))

In [84]:
print(train_positive_pairs_day1_server.take(2))

[('221.179.18.179', 'day1-8618761696532-221.179.18.179'), ('221.179.19.156', 'day1-8613851906441-221.179.19.156')]


In [85]:
all_service = list(set(qq_5000_user_dpi_3days.map(lambda x: eval(x)[10]).collect()))
all_sub_service = list(set(qq_5000_user_dpi_3days.map(lambda x: eval(x)[11]).collect()))

In [87]:
print(len(all_service),len(all_sub_service))
print(all_service, all_sub_service)

14 41
['24', '18', '14', '10', '3', '2', '7', '0', '4', '23', '1', '8', '6', '21'] ['2091', '14', '782', '1844', '2087', '1035', '1544', '1049', '1029', '2346', '2171', '2573', '258', '1827', '2082', '337', '1043', '2088', '1793', '2148', '1545', '339', '515', '0', '1553', '1050', '2570', '2149', '323', '1326', '1073', '1025', '1042', '1322', '1041', '528', '1072', '322', '1074', '802', '777']


In [88]:
sc.parallelize(all_service).coalesce(1)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/all_service_ID")
sc.parallelize(all_sub_service).coalesce(1)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/all_sub_service_ID")

In [139]:
server_service = qq_5000_user_dpi_3days.map(lambda x: (eval(x)[2], eval(x)[10])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], dict(Counter(x[1]))))

In [140]:
print(server_service.take(2))
print(server_service.count())

[('14.17.19.77', {'4': 575, '7': 204}), ('111.10.27.180', {'4': 1, '6': 52})]
1684


In [141]:
def get_feature(dic, all_feature):
    res = [0 for i in range(len(all_feature))]
    #time_hour = ['00', '01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23']
    for i in range(len(all_feature)):
        res[i] = dic.get(all_feature[i],0)
    return res

In [142]:
server_service = server_service.map(lambda x:(x[0], get_feature(x[1], all_service)))

In [143]:
print(server_service.take(2))
print(server_service.count())

[('14.17.19.77', [0, 0, 0, 0, 0, 0, 204, 0, 575, 0, 0, 0, 0, 0]), ('111.10.27.180', [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 52, 0])]
1684


In [144]:
server_service.coalesce(1)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_service_feature")

In [145]:
server_sub_service = qq_5000_user_dpi_3days.map(lambda x: (eval(x)[2], eval(x)[11])).groupByKey().mapValues(list)\
.map(lambda x: (x[0], dict(Counter(x[1]))))

In [146]:
print(server_sub_service.take(2))

[('14.17.19.77', {'1025': 575, '1793': 204}), ('111.10.27.180', {'1025': 1, '1029': 52})]


In [147]:
server_sub_service = server_sub_service.map(lambda x:(x[0], get_feature(x[1], all_sub_service)))

In [148]:
print(server_sub_service.take(2))

[('14.17.19.77', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 204, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 575, 0, 0, 0, 0, 0, 0, 0, 0, 0]), ('111.10.27.180', [0, 0, 0, 0, 0, 0, 0, 0, 52, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


In [149]:
server_sub_service.coalesce(1)\
.saveAsTextFile("/media/jw/5dccc50e-7c13-4186-bf6b-d894f02410be/wbb/result/pu_learning/server_subservice_feature")